In [ ]:
# Note: Tensor Network does not make sense in this problem.

In [5]:
import json

import numpy as np 
import tensornetwork as tn 
import tensorflow as tf 
from tqdm import tqdm

from mlexpt.data.adding_features import adding_no_features, convert_label_to_str
from mlexpt.experiment import add_multiple_features
from mlexpt.experiment import run_experiment
from mlexpt.ml.core import ExperimentalClassifier

In [2]:
class QuantumDMRGLayer(tf.keras.layers.Layer):
    def __init__(self, dimvec, pos_label, nblabels, bond_len, nearzero_std=1e-9, isolated_labelnode=True):
        super(QuantumDMRGLayer, self).__init__()
        self.dimvec = dimvec
        self.pos_label = pos_label
        self.nblabels = nblabels
        self.m = bond_len
        self.isolated_label = isolated_labelnode

        assert self.pos_label >= 0 and self.pos_label < self.dimvec

        self.mps_tensors = [tf.Variable(self.mps_tensor_initial_values(i, nearzero_std=nearzero_std),
                                        trainable=True,
                                        name='mps_tensors_{}'.format(i))
                            for i in range(self.dimvec)]
        if self.isolated_label:
            self.output_tensor = tf.Variable(tf.random.normal((self.m, self.m, self.nblabels),
                                                              mean=0.0,
                                                              stddev=nearzero_std),
                                             trainable=True,
                                             name='mps_output_node')

    def mps_tensor_initial_values(self, idx, nearzero_std=1e-9):
        if idx == 0 or idx == self.dimvec - 1:
            tempmat = tf.eye(max(2, self.m))
            mat = tempmat[0:2, :] if 2 < self.m else tempmat[:, 0:self.m]
            return mat + tf.random.normal(mat.shape, mean=0.0, stddev=nearzero_std)
        elif not self.isolated_label and idx == self.pos_label:
            return tf.random.normal((2, self.m, self.m, self.nblabels),
                                    mean=0.0,
                                    stddev=nearzero_std)
        else:
            return tf.random.normal((2, self.m, self.m),
                                    mean=0.0,
                                    stddev=nearzero_std)

    def infer_single(self, input):
        assert input.shape[0] == self.dimvec
        assert input.shape[1] == 2

        nodes = [
            tn.Node(self.mps_tensors[i], backend='tensorflow')
            for i in range(self.dimvec)
        ]
        if self.isolated_label:
            output_node = tn.Node(self.output_tensor, backend='tensorflow')
        input_nodes = [
            tn.Node(input[i, :], backend='tensorflow')
            for i in range(self.dimvec)
        ]

        for i in range(self.dimvec):
            nodes[i][0] ^ input_nodes[i][0]
        if self.isolated_label:
            nodes[0][1] ^ nodes[1][1]
            for i in range(1, self.pos_label):
                nodes[i][2] ^ nodes[i + 1][1]
            nodes[self.pos_label][2] ^ output_node[0]
            output_node[1] ^ nodes[self.pos_label + 1][1]
            for i in range(self.pos_label + 1, self.dimvec - 1):
                nodes[i][2] ^ nodes[i + 1][1]
        else:
            nodes[0][1] ^ nodes[1][1]
            for i in range(1, self.dimvec-1):
                nodes[i][2] ^ nodes[i + 1][1]

        if self.isolated_label:
            final_node = tn.contractors.auto(nodes + input_nodes + [output_node],
                                             output_edge_order=[output_node[2]])
        else:
            final_node = tn.contractors.auto(nodes + input_nodes,
                                             output_edge_order=[nodes[self.pos_label][3]])
        return final_node.tensor

    def call(self, inputs):
        return tf.vectorized_map(self.infer_single, inputs)

In [3]:
import json

configs = json.loads("""
{
  "model": {
    "qualitative_features": ["color", "shape"],
    "binary_features": ["ripe"],
    "quantitative_features": ["radius", "weight"],
    "target": "fruit",
    "embedding_schemes": {
    },
    "algorithm": "LogisticRegression",
    "model_parameters": {
      "device": "cpu",
      "nb_epoch": 100
    }
  },
  "data": {
    "path": "exampledataset/fruits.json",
    "missing_value_filling": {},
    "torchdevice": "cpu"
  },
  "train": {
    "cross_validation": true,
    "cv_nfold": 5,
    "heldout_fraction": 0.05,
    "persist_model": false,
    "model_path": null
  },
  "statistics": {
    "topN": 2,
    "compute_class_performance": true,
    "class_performances_excel": null
  }
}
""")

In [4]:
configs

{'model': {'qualitative_features': ['color', 'shape'],
  'binary_features': ['ripe'],
  'quantitative_features': ['radius', 'weight'],
  'target': 'fruit',
  'embedding_schemes': {},
  'algorithm': 'LogisticRegression',
  'model_parameters': {'device': 'cpu', 'nb_epoch': 100}},
 'data': {'path': 'exampledataset/fruits.json',
  'missing_value_filling': {},
  'torchdevice': 'cpu'},
 'train': {'cross_validation': True,
  'cv_nfold': 5,
  'heldout_fraction': 0.05,
  'persist_model': False,
  'model_path': None},
 'statistics': {'topN': 2,
  'compute_class_performance': True,
  'class_performances_excel': None}}

In [ ]:
class QuantumFruitClassifier(ExperimentalClassifier):
    def __init__(self):
        self.kerasmodel = None
        self.trained = False
    
    def fit(self, X, Y):
        pass